In [1]:
import numpy
from scipy import sparse

class Network:
    """ Construct a recurrent neural network (based on the Sussillo implementation cited
        in the README). The network is described by time_constant, or the time steps
        in the overall evolution of the network, num_neurons, or the number of neurons
        in the network, chaotic_constant, or the stabilizing weighting constant that
        allows for chaotic activity in the network.
        
        [...explain other input variables...]
        readout_sparseness = p_z in paper
        g_z is an output constant scaling factor
    """
    def __init__(self, time_constant, num_neurons, p, chaotic_constant, input_num, 
                 output_num, gg_sparseness, gz_sparseness, fg_sparseness, readout_sparseness,
                 g_gz, dt):
        # evolution time constant
        self.time_constant = time_constant
        
        # number of neurons in the network
        self.num_neurons = num_neurons
        
        # chaotic constant. Sussillo recommends chaotic_constant = 1.5 to allow for chaotic dynamics
        self.chaotic_constant = chaotic_constant
        
        # membrane potentials of each neurons (what should this be initialized to?)
        self.membrane_potential = 0.5 * numpy.random.rand(num_neurons) # neurons
        print("array of zeros supposedly", self.membrane_potential)
        
        # number of input neurons
        self.input_num = input_num
        
        # number of output neurons
        self.output_num = output_num
        
        # Connectivity matrix dictating synaptic strength of connections between neurons 
        # in the network. (J^{GG} in the paper), gg_sparseness is inter-neuron sparsenesss.
        scale = 1.0 / numpy.sqrt(p * num_neurons)
        self.connectivity_matrix = scale * chaotic_constant * sparse.rand(num_neurons, num_neurons, density=(1-gg_sparseness)).toarray()
        
        # Connectivity matrix dictating synaptic strength of connections between neurons
        # in the network and readout neurons
        self.feedout_matrix = sparse.rand(output_num, num_neurons, density=(1-gz_sparseness)).toarray()
        
        # Connectivity matrix dictating synaptic strength of connections between neurons
        # in the network and feedin neurons
        self.feedin_matrix = sparse.rand(input_num, num_neurons, density=(1-fg_sparseness)).toarray()
        
        # the vector w in Sussillo
        self.readout_matrix = sparse.rand(output_num, 1, density=(1-readout_sparseness)).toarray()
        
        # r(t)
        self.network_activities = sparse
        
        # J^{G_z} (unsure of dimension here)
        self.z_matrix = 2 * (numpy.random.rand(num_neurons, 1) - 0.5)
        
        # scaling factor for z
        self.g_gz = g_gz
        self.dt = dt
        
    """ Propogate the feedback input in z through the network. This models the differential
        equation presented in Sussillo. z is the dot product of w and r(t) (see definitions above)
    """
    def prop(self, z, r):
        for neuron in range(self.num_neurons):
            g_GG = self.chaotic_constant
            x_i = self.membrane_potential[neuron]
            
            y_0 = 0
            for n in range(self.num_neurons):
                y_0 += self.connectivity_matrix[neuron][n] * r[n] # numpy.tanh(self.membrane_potential[n])
            
            # first term in network equation
            # y_0 *= g_GG
            
            # second term
            y_1 = self.g_gz * self.z_matrix[neuron] * z * dt
            self.membrane_potential[neuron] = (1.0 - self.time_constant) * x_i + y_0 * dt + y_1

"""Start with 100 neurons. membrane_potential[i] is the membrane potential of
   neuron i."""
# network = Network(0.001, 1000, 1.7, 10, 10, 0.9, 0.9, 0.9, 0.1, 0.2)
# network.prop(0.009)
# print(type(network.connectivity_matrix))
# print(network.membrane_potential[:])


'Start with 100 neurons. membrane_potential[i] is the membrane potential of\n   neuron i.'